# Onboard a model defined in a .wf1 file
This notebook takes a .wf1 workfile and transform  it to a modelflow model.<br>

The overall process is performed in a Dataclass named:**GrapWfModel**<br> 
Close study of this class can be rewarding, but is outside the need of most users.<br>
The overall structure is: 
 1. Eviews is started and the wf1 (or wf2) file is loaded
    -    Some transformations are performed on data.
    -    The model is unlinked 
    -    The workspace is saved as a wf2 file. Same name with ```_modelflow appended```
 5. Eviews is closed 
 6. The wf2 file is read as a json file. 
 7. Relevant objects are extracted. 
 7. The MFMSA variable is  extracted, to be saved in the dumpfile. 
 8. The equations are transformed and normalized to modelflow format and classified into identities and stochastic
 9. Stochastic equations are enriched by add_factor and fixing terms (dummy + fixing value)  
 9. For Stochastic equations new fitted variables are generated - without add add_factors and dummies.  
 9. A model to generate fitted variables is created  
 9. A model to generate add_factors is created. 
 9. A model encompassing the original equations, the model for fitted variables and for add_factors is created. 
 9. The data series and scalars are shoveled into a Pandas dataframe 
     - Some special series are generated as the expression can not be incorporated into modelflow model specifications
     - The model for fitted values is simulated in the specified timespan
     - The model for add_factors is simulated in the timespan set in MFMSA
 10. The data descriptions are extracted into a dictionary. 
    - Data descriptions for dummies, fixed values, fitted values and add_factors are derived. 
 11. Now we have a model and a dataframe with all variables which are needed.
 b
The GrapWfModel instance in general keeps most of the steps so the developer can inspect the the different steps.  

## Prerequisites  

- Eviews version 12 
- The python library: ```pyevies```  


In [28]:
%matplotlib Notebook

In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
from pathlib import Path

from modelclass import model
from modelgrabwf2 import GrabWfModel
model.widescreen()

In [31]:
#longname='KENsoln20230410useTHIS_modelflow'
longname='KENmod20230209elec2'
filename = Path(f'data/{longname}.wf2')              
modelname = 'ken'

## Model specific transformations 
Not all Eviews equations have a direct counterpart in modelflow.<br>
To handle that two types of transformations can be provided for a specific model. 
- **eviews_run_lines** which specifies a list of Eviews command lines to be run. Typical to make some transformation of data which is not possible in modelflow. 
- **country_trans** which specifies a list of replacements to be done to the Eviews formulas before they are processed to modelflow. Typical to use the transformation mentioned above.  

## Process the model

In [32]:
all_about_ken = GrabWfModel(filename, 
                    make_fitted = True,       # If we want to calculate a fittet value for stocastic eq 
                    do_add_factor_calc=True,   # Stocastic equations are argumented by add factor 
                    start = 2020,
                    end = 2050, 
                    fit_start = 2000,          # Start of calculation of fittet model in baseline 
                    fit_end   = 2050           # end of calc for fittted model, if None taken from mdmfsa options  
                    ) 


Reading c:\wb ftt\code\FTT-ModelFlow\FTT_modelflow\models\presentation 20230413\data\KENmod20230209elec2.wf2
Assummed model name: KEN
The model: KEN is unlinked 
Writing C:\wb ftt\code\FTT-ModelFlow\FTT_modelflow\models\presentation 20230413\data\KENmod20230209elec2_modelflow.wf2
Model name: KEN

Processing the model:KEN
Estimated coifficients are substituted


Normalizing model: 100%|███████████████████████████████████████████████████████████████████████████████████████|187/187

No quasiIdentities in KEN
Default WB var_group loaded
Variable description in wf1 file read
Default WB var_description loaded
testmodel calculated  
Calculation of add factors for KEN calculated  


## Check if each equation on its own result in the values provided. 
aka: residual check <br> 
If they are not pretty close, something is very wrong. 

In [33]:
all_about_ken.test_model(all_about_ken.start,all_about_ken.end,maxerr=100,tol=0.001,showall=0)

KEN calculated  

Chekking residuals for KEN 2020 to 2050


## Extract the model and the baseline
From **all_about_ken** the model and baseline can be extracted

In [34]:
mken,baseline      = all_about_ken()

## Look a all the modelflow frmls
Notice after the "original" model the equations for the "fitted" values have been added. <br>
Also in the end of the listing the specification of the model which calculates the add factors if a variable is fixed. When processing the equations the ```model``` class will process this this model separately and create a model instance 
which is used to calculate add factors in case 

In [35]:
#print(mken.equations)

## Run the model 
In order to achieve numerical stability Gauss-Seidle has to be dampened: alfa=0.7 makes sure that the solution does not explode. 
The convergence criteria is tightend a lot.   

In [36]:
res = mken(baseline,2022,2100,silent=1)

## And inspect the simulation result. 
Notice that the variable descriptions (from the generic WB naming are used. 

In [37]:
mken['*GGEXPCAPTCN *NYGDPMKTPCN *GGDBTTOTLCN *BNCABFUNDCD KENTOTELECQN']

In [38]:
mken[['!*GDP*per*']].endo

## Look at a stochastic variable 
Here the equations undergo more phases 

In [39]:
all_about_ken.all_frml_dict['KENGGEXPCAPTCN'].fprint


Endo_var        : KENGGEXPCAPTCN
Original        : DLOG(KENGGEXPCAPTCN)=-0.169741292945079*(LOG(KENGGEXPCAPTCN(-1))-LOG(KENNYGDPMKTPCN(-1))-LOG(0.0695471984012515))+0.178981111257331*DLOG(KENGGEXPCAPTCN(-1))+(1-0.178981111257331)*DLOG(KENNYGDPMKTPCN)
Preprocessed    : ((LOG(KENGGEXPCAPTCN))-(LOG(KENGGEXPCAPTCN(-1))))=-0.169741292945079*(LOG(KENGGEXPCAPTCN(-1))-LOG(KENNYGDPMKTPCN(-1))-LOG(0.0695471984012515))+0.178981111257331*((LOG(KENGGEXPCAPTCN(-1)))-(LOG(KENGGEXPCAPTCN(-2))))+(1-0.178981111257331)*((LOG(KENNYGDPMKTPCN))-(LOG(KENNYGDPMKTPCN(-1))))
Normalized      : KENGGEXPCAPTCN = (KENGGEXPCAPTCN(-1)*EXP(KENGGEXPCAPTCN_A+ (-0.169741292945079*(LOG(KENGGEXPCAPTCN(-1))-LOG(KENNYGDPMKTPCN(-1))-LOG(0.0695471984012515))+0.178981111257331*((LOG(KENGGEXPCAPTCN(-1)))-(LOG(KENGGEXPCAPTCN(-2))))+(1-0.178981111257331)*((LOG(KENNYGDPMKTPCN))-(LOG(KENNYGDPMKTPCN(-1))))) )) * (1-KENGGEXPCAPTCN_D)+ KENGGEXPCAPTCN_X*KENGGEXPCAPTCN_D 
Calc_add_factor : KENGGEXPCAPTCN_A = - ((-0.169741292945079*(LOG(

In [82]:
mken['KENGGEXPCAP*'].eviews

KENGGEXPCAPTCN        : DLOG(KENGGEXPCAPTCN) =- 0.169741292945079*(LOG(KENGGEXPCAPTCN( - 1)) - LOG(KENNYGDPMKTPCN( - 1)) - LOG(0.0695471984012515)) + 0.178981111257331*DLOG(KENGGEXPCAPTCN( - 1)) + (1 - 0.178981111257331)*DLOG(KENNYGDPMKTPCN)
KENGGEXPCAPTCN_A      : Exogen
KENGGEXPCAPTCN_D      : Exogen
KENGGEXPCAPTCN_FITTED : Not avaible
KENGGEXPCAPTCN_X      : Exogen


In [49]:
mken.KENGGEXPCAPTCN.eviews

DLOG(KENGGEXPCAPTCN) =- 0.169741292945079*(LOG(KENGGEXPCAPTCN( - 1)) - LOG(KENNYGDPMKTPCN( - 1)) - LOG(0.0695471984012515)) + 0.178981111257331*DLOG(KENGGEXPCAPTCN( - 1)) + (1 - 0.178981111257331)*DLOG(KENNYGDPMKTPCN)


In [50]:
mken.KENGGEXPCAPTCN_FITTED.eviews

Not avaiable


In [118]:
mken['*KEN*GDP*'].endo.eviews

KENENTESGDPINTENSITY        : DLOG(KENENTESGDPINTENSITY) = -0.648987106406022*DLOG(KENNYGDPMKTPKN) + 0.0219112133714127
KENENTESGDPINTENSITY_FITTED : Not avaible
KENGDPPCKD                  : @IDENTITY KENGDPPCKD  = (KENNYGDPMKTPKD  / KENSPPOPTOTL)
KENGDPPCKN                  : @IDENTITY KENGDPPCKN  = (KENNYGDPMKTPKN  / KENSPPOPTOTL)
KENNYGDPFCSTCN              : @IDENTITY KENNYGDPFCSTCn  = KENNYGDPFCSTXN  * KENNYGDPFCSTKN
KENNYGDPFCSTKN              : @IDENTITY KENNYGDPFCSTkn  = KENNYGDPMKTPKN  - KENNYTAXNINDKN
KENNYGDPFCSTXN              : DLOG(KENNYGDPFCSTXN) = 0.8*(0.4*LOG(1 + KENINFLEXPT/100) + (1 - 0.4)*DLOG(KENPSTAR)) + 1*KENNYGDPGAP_/100 + (1 - 0.8)*DLOG(KENNYGDPFCSTXN( - 1)) + 0.018986373422419*@DURING("2015")
KENNYGDPFCSTXN_FITTED       : Not avaible
KENNYGDPGAP_                : @IDENTITY KENNYGDPGAP_  = ((KENNYGDPMKTPKN  / KENNYGDPPOTLKN  - 1)  * 100)
KENNYGDPMKTPCD              : @IDENTITY KENNYGDPMKTPCD  = KENNYGDPMKTPCN  / KENPANUSATLS
KENNYGDPMKTPCN              : @IDEN

In [120]:
mken['#Headline']

In [61]:
with mken.set_smpl(2020,2023):
    print(mken.kenGGEXPCAPTCN.show)                        

Endogeneous: KENGGEXPCAPTCN: General government expenditure on capital expenditure (millions lcu)
Formular: FRML <DAMP,STOC> KENGGEXPCAPTCN = (KENGGEXPCAPTCN(-1)*EXP(KENGGEXPCAPTCN_A+ (-0.169741292945079*(LOG(KENGGEXPCAPTCN(-1))-LOG(KENNYGDPMKTPCN(-1))-LOG(0.0695471984012515))+0.178981111257331*((LOG(KENGGEXPCAPTCN(-1)))-(LOG(KENGGEXPCAPTCN(-2))))+(1-0.178981111257331)*((LOG(KENNYGDPMKTPCN))-(LOG(KENNYGDPMKTPCN(-1))))) )) * (1-KENGGEXPCAPTCN_D)+ KENGGEXPCAPTCN_X*KENGGEXPCAPTCN_D  $

KENGGEXPCAPTCN  : General government expenditure on capital expenditure (millions lcu)
KENGGEXPCAPTCN_A: Add factor:General government expenditure on capital expenditure (millions lcu)
KENGGEXPCAPTCN_D: Fix dummy:General government expenditure on capital expenditure (millions lcu)
KENGGEXPCAPTCN_X: Fix value:General government expenditure on capital expenditure (millions lcu)
KENNYGDPMKTPCN  : GDP, Market Prices, LCU mn

Values :


,2020,2021,2022,2023
Base,"761,102.36","831,143.45","913,757.45","1,050,444.32"
Last,"761,102.36","831,143.45","913,757.45","1,050,444.32"
Diff,0.00,0.00,-0.00,-0.00


Input last run:


,2020,2021,2022,2023
KENGGEXPCAPTCN(-1),"707,006.38","761,102.36","831,143.45","913,757.45"
KENGGEXPCAPTCN(-2),"588,726.62","707,006.38","761,102.36","831,143.45"
KENGGEXPCAPTCN_A,0.00,-0.02,-0.02,-0.01
KENGGEXPCAPTCN_D,0.00,0.00,0.00,0.00
KENGGEXPCAPTCN_X,0.00,0.00,0.00,0.00
KENNYGDPMKTPCN,"10,716,034.00","12,098,175.51","13,546,261.09","15,854,442.43"
KENNYGDPMKTPCN(-1),"10,237,727.00","10,716,034.00","12,098,175.51","13,546,261.09"


Input base run:


,2020,2021,2022,2023
KENGGEXPCAPTCN(-1),"707,006.38","761,102.36","831,143.45","913,757.45"
KENGGEXPCAPTCN(-2),"588,726.62","707,006.38","761,102.36","831,143.45"
KENGGEXPCAPTCN_A,0.00,-0.02,-0.02,-0.01
KENGGEXPCAPTCN_D,0.00,0.00,0.00,0.00
KENGGEXPCAPTCN_X,0.00,0.00,0.00,0.00
KENNYGDPMKTPCN,"10,716,034.00","12,098,175.51","13,546,261.09","15,854,442.43"
KENNYGDPMKTPCN(-1),"10,237,727.00","10,716,034.00","12,098,175.51","13,546,261.09"


Difference for input variables


,2020,2021,2022,2023
KENGGEXPCAPTCN(-1),0.00,0.00,0.00,-0.00
KENGGEXPCAPTCN(-2),0.00,0.00,0.00,0.00
KENGGEXPCAPTCN_A,0.00,0.00,-0.00,0.00
KENGGEXPCAPTCN_D,0.00,0.00,0.00,0.00
KENGGEXPCAPTCN_X,0.00,0.00,0.00,0.00
KENNYGDPMKTPCN,0.00,0.00,-0.00,-0.00
KENNYGDPMKTPCN(-1),0.00,0.00,0.00,-0.00



None


## Now dump the model and data 

In [ ]:
mken.model_description=f'Keny a model from {longname}'
mken.modeldump(f'pcim/{longname}.pcim')

In [ ]:
try:
     mken.modeldump(f'C:/modelflow manual/model_repo/{longname}.pcim')
except: 
     print('could not write to modelrepo file')